In [20]:
import json
import requests
import pandas as pd
import base64
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize

#### Web Scraping

In [42]:
#Publicacion resultados: https://es.investing.com/equities/apple-computer-inc-earnings
url='https://es.investing.com/equities/apple-computer-inc-earnings'
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}

#https://es.investing.com/equities/morehistory

html=requests.get(url,headers=headers).content
soup=BeautifulSoup(html,'lxml')

In [63]:
table = soup.find_all('table',{'id':'earningsHistory6408'})

In [64]:
table2=table[0]

In [69]:
rows=table2.find_all('tr')
rows = [row.text.strip().split('\n') for row in rows]
rows =[list(filter(None,row)) for row in rows]

In [68]:
colnames = ['Fecha de publicación','Período terminado','BPA','Previsión','Ingresos','Previsión']
datos=rows
df=pd.DataFrame(datos[1:], columns=colnames)
df

,Fecha de publicación,Período terminado,BPA,Previsión,Ingresos,Previsión
0,05.05.2020,03/2020,--,"/ 2,99",--,"/ 65,25B"
1,28.01.2020,12/2019,"4,99","/ 4,54","91,82B","/ 88,51B"
2,30.10.2019,09/2019,"3,03","/ 2,83","64,04B","/ 63,01B"
3,30.07.2019,06/2019,"2,18","/ 2,09","53,81B","/ 53,32B"
4,30.04.2019,03/2019,"2,46","/ 2,36","58,02B","/ 57,51B"
5,29.01.2019,12/2018,"4,18","/ 4,17","84,31B","/ 83,97B"


### API

In [14]:
#https://financialmodelingprep.com/developer/docs/companies-key-stats-free-api/#Python

#'https://financialmodelingprep.com/api/v3/historical-price-full/AAPL'


url = 'https://financialmodelingprep.com/api/v3/historical-price-full/AAPL'
response = requests.get(url)



In [15]:
result=response.json()
result
data=pd.DataFrame(json_normalize(result))

In [16]:
data['historical'][0]

,symbol,historical
0,AAPL,"[{'date': '2015-02-17', 'open': 127.49, 'high'..."


In [17]:
data2=json_normalize(data['historical'][0])
data2

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime
0,2015-02-17,127.49,128.88,126.92,127.83,118.27,63152400.0,63152400.0,-0.34,-0.267,127.87667,"February 17, 15",-0.00267
1,2015-02-18,127.63,128.78,127.45,128.72,119.09,44891700.0,44891700.0,-1.09,-0.854,128.31667,"February 18, 15",-0.00854
2,2015-02-19,128.48,129.03,128.33,128.45,118.84,37362400.0,37362400.0,0.03,0.023,128.60333,"February 19, 15",0.00023
3,2015-02-20,128.62,129.50,128.05,129.50,119.82,48948400.0,48948400.0,-0.88,-0.684,129.01667,"February 20, 15",-0.00684
4,2015-02-23,130.02,133.00,129.66,133.00,123.05,70974100.0,70974100.0,-2.98,-2.292,131.88667,"February 23, 15",-0.02292
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,2020-02-10,314.18,321.55,313.85,321.55,321.55,27337200.0,27337200.0,-7.37,-2.346,318.98333,"February 10, 20",-0.02346
1255,2020-02-11,323.60,323.90,318.71,319.61,319.61,23580800.0,23580800.0,3.99,1.233,320.74000,"February 11, 20",0.01233
1256,2020-02-12,321.47,327.22,321.47,327.20,327.20,28432600.0,28432600.0,-5.73,-1.782,325.29667,"February 12, 20",-0.01782
1257,2020-02-13,324.19,326.22,323.35,324.87,324.87,23686900.0,23686900.0,-0.68,-0.210,324.81333,"February 13, 20",-0.00210
